In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import time
from datetime import datetime, timedelta
import os
import itertools

from fetch_api import entsoe_api, energinet_api
from functions import *
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Add the SoC/MWh model of the boiler or Load the complete csv schedule containing MWh 

Add aFRR activation prices, Spot prices from Energinet API

In [47]:
# Define the duration of the data to be fetched. The bids are compared against the aFRR price during the period assuming complete visibility

start_date = '2024-11-26'
end_date = '2024-12-03'

In [26]:
"""
Fetch aFRR prices using energinet api
"""

aFRRprice = energinet_api('AfrrEnergyActivated', start_date, end_date)
aFRRprice = aFRRprice[aFRRprice['PriceArea'] == 'DK1']
aFRRprice['ActivationTime'] = pd.to_datetime(aFRRprice['ActivationTime'])
aFRRprice = aFRRprice.sort_values('ActivationTime', ascending=True).reset_index(drop=True)
aFRRprice['Interval'] = (aFRRprice['ActivationTime'].dt.hour * 4 + aFRRprice['ActivationTime'].dt.minute // 15 + 1)
aFRRprice['MTU_block'] = aFRRprice['ActivationTime'].dt.floor('15T') # sort all rows to every 15 min bucket
aFRRpriceMTU = aFRRprice.groupby('MTU_block', as_index=False).apply(lambda x: x.iloc[[0]]).reset_index(drop=True) # ensures if aFRR is missing in the first row of the MTU, returns nan

In [48]:
"""
Fetch Spot Prices using energinet api
The lowest permissible bid price = spot price
"""

spotpricesthisweek = energinet_api('Elspotprices', start_date, end_date)
spotpricesthisweek = spotpricesthisweek.loc[spotpricesthisweek['PriceArea'] == 'DK1']
spotpricesthisweek['HourUTC'] = pd.to_datetime(spotpricesthisweek['HourUTC'])
spotpricesthisweek.index = range(0, len(spotpricesthisweek))
spotpricesthisweek.set_index('HourUTC', inplace=True)
spotpricesthisweek = spotpricesthisweek.resample('15T').ffill()

In [ ]:
"""
The reference LinHeat schedule for the week 
"""
folder_path = r"D:\ms\January 2024\Thesis\Boiler-Bidding-model\Datasets\LinHeatSchedule_1week_estimatedMWh.csv"
commitment_schedule = pd.read_csv(folder_path)
commitment_schedule['UTC'] = pd.to_datetime(commitment_schedule['UTC'])

# shift the commitment schedule to match the prices fetched dates
time_shift = pd.Timestamp(start_date, tz= 'UTC') - commitment_schedule['UTC'].min()
commitment_schedule['UTC'] = commitment_schedule['UTC'] + time_shift 
commitment_schedule.set_index('UTC', inplace= True)
# merge the spot prices of each hour to the commitment_schedule
commitment_schedule.index = commitment_schedule.index.tz_localize(None)
commitment_schedule = commitment_schedule.join(spotpricesthisweek, how="left")
commitment_schedule.reset_index(inplace=True)
commitment_schedule.rename(columns={'index': 'UTC'}, inplace=True)

Set up datasets for the model

In [53]:
"""
All required data is assigned to arrays
"""
commitment = np.array(commitment_schedule['Total heat production plan [MW]']) # this column is changed to L11 EK Plan (MW)
spotpricesoftheday = np.array(commitment_schedule['SpotPriceEUR'])
historical_up_clearing_price = np.array(aFRRpriceMTU['aFRR_UpActivatedPriceEUR'])
historical_down_clearing_price = np.array(aFRRpriceMTU['aFRR_DownActivatedPriceEUR'])
scheduled_soc = np.array(commitment_schedule['estimate_soc'])
storage = np.array(commitment_schedule['Heat Storage (MW)'])
production_sch = np.array(commitment_schedule['Total heat production plan [MW]'])
estimated_soc = scheduled_soc.copy()


In [56]:
"""
Set up the dataframe to save the output results
"""
simulation_time = len(commitment_schedule) - 2
results2 = pd.DataFrame(index=range(simulation_time + 2), columns=['final_soc', 'win_outcome', 'planned_up_bid_price', 'planned_down_bid_price',
    'planned_up_qty', 'planned_down_qty', 'actual_up_activation', 'actual_down_activation'])
results2[:] = np.nan 
results2['UTC'] = np.array(commitment_schedule['UTC'])
results2.set_index('UTC', inplace= True)
results2.reset_index(inplace=True)
for t in range(2):
    results2.loc[t ,'final_soc'] = scheduled_soc[0]
results2 =  results2.merge(commitment_schedule[['UTC', 'Total heat production plan [MW]', 'Heat Storage (MW)', 'SpotPriceEUR', 'estimate_soc']], on='UTC', how='left')
results2 =  results2.merge(aFRRpriceMTU[['ActivationTime','aFRR_UpActivatedPriceEUR', 'aFRR_DownActivatedPriceEUR']], left_on='UTC', right_on='ActivationTime', how='left').drop(columns=['ActivationTime'])
results2.index = range(-1, -1 + len(results2))  # D 00:00 should be index 0 and anything before that MTU is -ve index

In [ ]:
# max ch and disch rate
Pch =  9.645000000000001 
Pdis =  -4.429565217391305
Emax = 60
# define the model parameters
simulation_time = len(commitment_schedule) - 2
lead_time = 2 # bid submission leads the outcome by 2 MTUs
start_time = datetime.strptime(start_date, "%Y-%m-%d")
for t in range(simulation_time + 2):
    current_time = start_time + timedelta(minutes=15 * t)
    print(f'Simulation at {current_time}')
    finalize_index = t - lead_time # at simulation time t, finalise the bids after 2 MTUs in the future
    decision_index = t + lead_time # at simulation time t, we know the outcome of (t-2) MTUs

    # if t == 0 or t == 1:
    #     # we dont need to make any soc finalization, no old outcomes

    if finalize_index >= 0: # we find outcome of the past MTU; at t = 2 (00:30), outcome of t = 0 (00:00) is known, finalize our state of the system
        print(f'Finalizing outcome of MTU:{finalize_index} at {current_time - timedelta(minutes=15 * 2)}')
        up_bid_price = results2.loc[finalize_index, 'planned_up_bid_price'] # retrieve the prices we bid for this MTU 25 mins ago
        down_bid_price = results2.loc[finalize_index, 'planned_down_bid_price']

        if up_bid_price <= historical_up_clearing_price[finalize_index] and down_bid_price >= historical_down_clearing_price[finalize_index]:
            actual_win = 'BOTH'
        elif up_bid_price <= historical_up_clearing_price[finalize_index]:
            actual_win = 'UP'
        elif down_bid_price >= historical_down_clearing_price[finalize_index]:
            actual_win = 'DOWN'
        else:
            actual_win = 'NIL'
        
        results2.loc[finalize_index, 'win_outcome'] = actual_win # update if we have won the MTU

        # once we know the win status, determine the activation rate during the MTU placed 30 mins ago
        if actual_win in ['UP', 'DOWN', 'BOTH']:
            if actual_win == 'BOTH':
                alpha_up_val = activation_estimate()
                alpha_down_val = activation_estimate()
                results2.loc[finalize_index, 'actual_up_activation'] = alpha_up_val
                results2.loc[finalize_index, 'actual_down_activation'] = alpha_down_val
            else:
                alpha = activation_estimate()
                if actual_win == 'UP':
                    alpha_up_val = alpha
                    results2.loc[finalize_index, 'actual_up_activation'] = alpha
                else:
                    alpha_down_val = alpha
                    results2.loc[finalize_index, 'actual_down_activation'] = alpha
        # with the activation rate, determine the actual soc
        soc = results2.loc[finalize_index, 'final_soc']
        if actual_win == 'UP':
            current_soc = update_soc(soc, Emax, -upbidsize, alpha_up_val)
        elif actual_win == 'DOWN':
            current_soc = update_soc(soc, Emax, downbidsize, alpha_down_val)
        elif actual_win == 'BOTH':
            net_change = (downbidsize * alpha_down_val) - (upbidsize * alpha_up_val)
            current_soc = update_soc(soc, Emax, net_change, 1.0)
        else:
            current_soc = soc_on_schedule(Pch, Pdis, soc, storage[finalize_index], production_sch[finalize_index], Emax, Pmax, wch = 1.2, wdis = 1)

        results2.loc[finalize_index + 1, 'final_soc'] = current_soc # the soc at the end of the finalized MTU goes into the entry of next MTU
        print(f'Bid Status for MTU {finalize_index}: {actual_win} and SoC changes {np.round(soc,2)}:MTU[{finalize_index}] --> {np.round(current_soc,2)}:MTU[{finalize_index+1}]')


    if decision_index < simulation_time: # we find the the bids for MTU in the future; t = 0 (00:00), bids for t = 2 (00:30)  is decided
        # for t = 0 and t = 1, we don't have any history, use estimated soc
        print(f'Calculating bids for MTU:{decision_index} at {current_time + timedelta(minutes=15 * 2)}')

        if t <= 1:
            soc_for_bidding = results2.loc[finalize_index+1, 'estimate_soc']
            results2.loc[finalize_index + 1, 'final_soc'] = soc_for_bidding
        else:
            soc_for_bidding = results2.loc[finalize_index+1, 'final_soc']

        up_bid_probability = bid_probability(soc_for_bidding, 'UP')
        down_bid_probability = bid_probability(soc_for_bidding, 'DOWN')

        upbidprice, downbidprice = bid_price(up_bid_probability, down_bid_probability, results2['SpotPriceEUR'], decision_index)
        results2.loc[decision_index, 'planned_up_bid_price'] = upbidprice
        results2.loc[decision_index, 'planned_down_bid_price'] = downbidprice

        upbidsize = bid_size(soc_for_bidding, Emax, 'UP')
        downbidsize = bid_size(soc_for_bidding, Emax, 'DOWN')
        results2.loc[decision_index, 'planned_up_qty'] = upbidsize # store the decisions for the MTU 2 steps in the future
        results2.loc[decision_index, 'planned_down_qty'] = downbidsize
    
        print(f'Bid placed in Up: {np.round(upbidsize,2)}Mwh for {np.round(upbidprice,2)}EUR and Down: {np.round(downbidsize,2)}Mwh for {np.round(downbidprice,2)}EUR for MTU {np.round(decision_index,2)}')
    print('*********************************')